# blah blah

In [1]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [96]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-28 16:14:24 [INFO] Started loading binary file
2019-10-28 16:14:24 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-28 16:14:24 [INFO] Finished loading binary file in 0.04s (0.06s CPU)


In [97]:
desc_col(ek_info)

,DTYPE,NULLS,UNIQUE
index,int64,0/32237,32237
Endkunde_NR,int64,0/32237,32237
Endkunde,object,0/32237,30291
EK_Aktiv,int64,0/32237,2
EK_Kam_Betreut,int64,0/32237,2
EK_Land,object,49/32188,45
PLZ,int64,0/32237,2875
GEMEINDE,object,49/32188,3006
Agentur,object,20999/11238,3247
EK_BG,object,1829/30408,83


# CRM STUFF

In [69]:
crm_data = load_bin("vkprog\\crm_data.feather").astype({"ENDKUNDE_NR": "int64"})

2019-10-28 14:34:23 [INFO] Started loading binary file
2019-10-28 14:34:23 [INFO] Reading from file C:\Users\stc\data\vkprog\crm_data.feather
2019-10-28 14:34:24 [INFO] Finished loading binary file in 0.21s (0.25s CPU)


In [71]:
display(crm_data.sample(5, random_state=42))
desc_col(crm_data)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
490351,Traffic SR-Actions 2017 - Changement d'adresse,E-Mail,2017-12-22,Marketing,Cédric Genoud (TRAFFIC ROMANDIE),CGE,486920,NaT,NaT,NaN
937554,Telemarketing,Besuch,2015-06-23,Verkauf,Massimo Bonotto (REGION OST CHUR),BON,604538,NaT,NaT,NaN
27740,2013_Weihnachtskarten - Mitte_2013,Brief,2013-11-22,Marketing,Maja Loosli (REGION VERKAUF MITTE),MAL,103253,NaT,NaT,NaN
799222,Ouest - Aktionen 2019 - Promotion d'été 2019 -...,E-Mail,2019-05-23,Marketing,Lisa Conte (REGION WEST GENF),LIC,570321,NaT,NaT,NaN
915179,Anfrage Buswerbung,E-Mail,2015-01-13,Verkauf,Domenico Torre (APG WINTERTHUR),DTO,598472,NaT,NaT,NaN


,DTYPE,NULLS,UNIQUE
BETREFF,category,26/1056374,336940
KANAL,category,0/1056400,12
STARTTERMIN,datetime64[ns],0/1056400,3817
QUELLE,category,0/1056400,3
VERANTWORTLICH,category,0/1056400,337
KUERZEL,category,1/1056399,340
ENDKUNDE_NR,int64,0/1056400,51555
VB_FILTER_VON,datetime64[ns],1054692/1708,13
VB_FILTER_BIS,datetime64[ns],1055145/1255,37
VB_FILTER_GRUND,category,1054709/1691,60


I need:
- ``Letzte_VBs``
- ``KZ_letzter_Ktkt`` (Letzter VB-Kontakt, Kürzel)
- ``Kanal`` (Kontaktweg: Email, Telefon, Brieftaube...)
- ``Betreff`` (Inhalt des letzten Kontakts)

In [6]:
crm_data.sort_values("STARTTERMIN", ascending=False).head(5)

,BETREFF,KANAL,STARTTERMIN,QUELLE,VERANTWORTLICH,KUERZEL,ENDKUNDE_NR,VB_FILTER_VON,VB_FILTER_BIS,VB_FILTER_GRUND
624924,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2024-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
69410,A contacter,Besuch,2023-10-21,Verkauf,Michel Zbinden (REGION WEST GENF),MZB,109933,NaT,NaT,NaN
624923,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2023-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624922,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2022-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN
624921,Kommunikationsmassnahmen 2020 - besprechen,E-Mail,2021-10-10,Verkauf,Roberto Pombar (APG|SGA AARAU),RPO,517178,NaT,NaT,NaN


In [81]:
def collect(s, sep=","):
    return sep.join(map(str, s[s.notna()].unique()))

In [38]:
def last_notna(s):
        try:
            return s.loc[s.notnull()].iat[-1]
        except IndexError:
            return np.NaN

In [93]:
def crm_info():
    today = pd.Timestamp.now()

    ## Information about last contact: Kanal, VB, Betreff
    row_select = (crm_data.loc[:,"STARTTERMIN"] <= today)
    container_crm = (crm_data.loc[row_select,:]
                             .sort_values(["ENDKUNDE_NR","STARTTERMIN"])
                             .groupby("ENDKUNDE_NR", as_index=False)
                             .agg(
                                 {"KUERZEL": last_notna,
                                  "KANAL": last_notna,
                                  "BETREFF": last_notna,
                                  #"STARTTERMIN": last_notna,
                                 }
                             )
                             .rename(columns={
                                 "KUERZEL": "Letzter_Kontakt",
                                 "KANAL": "Kanal",
                                 "BETREFF": "Betreff",
                                 "ENDKUNDE_NR": "Endkunde_NR",
                             })
                    )

    # Letzte CRM-Kontakte (alle der letzten zwei Kalendarjahre)
    row_select = (crm_data.loc[:,"STARTTERMIN"]
                          .apply(lambda x: 2 >= today.isocalendar()[0] - x.isocalendar()[0])
                  & (crm_data.loc[:,"STARTTERMIN"] <= today)
                 )

    crm_letzte_vbs = (crm_data.loc[row_select,:]
                              .groupby("ENDKUNDE_NR", as_index=False)
                              .agg(
                                  {"KUERZEL": collect,
                                  }
                              )
                              .rename(columns={
                                  "KUERZEL": "Letzte_CRM_Ktkts",
                                  "ENDKUNDE_NR": "Endkunde_NR",})
                     )
    # Combine
    container_crm = pd.merge(container_crm, crm_letzte_vbs, on="Endkunde_NR",how="left")

    return container_crm

In [94]:
ek_crm_info = crm_info()

In [95]:
display(ek_crm_info.head(5))
desc_col(ek_crm_info)

,Endkunde_NR,Letzter_Kontakt,Kanal,Betreff,Letzte_CRM_Ktkts
0,100034,IWI,E-Mail,Ost - Aktionen 2019 - ACO_Dispo_Eröffnung 1. H...,"HKL,ACO,BAC,LAH,BBO,JAU,LSC,IWI,MBU"
1,100039,IWI,E-Mail,Ost - Aktionen 2019 - CGA_Dispo_Eröffnung 1. H...,"DM,ROK,JDI,AKU,HUL,CGA,LSC,IWI"
2,100043,DM,E-Mail,Ost - Aktionen - 2017 - CGA_Dispoeröffnung 201...,"AKU,DM"
3,100045,ROK,E-Mail,Verträge Volkiland,"ACO,IWI,BAC,ROK,MBU,CNA,FIB,JAU,LSC,BAP,JEN"
4,100048,AKU,E-Mail,Ost - Aktionen - 2017 - CGA_Dispoeröffnung_1_2...,AKU


,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/51554,51554
Letzter_Kontakt,object,0/51554,269
Kanal,object,0/51554,11
Betreff,object,1/51553,10109
Letzte_CRM_Ktkts,object,10259/41295,13681


In [74]:
container_crm.sort_values("STARTTERMIN",ascending=False).head(5)

,ENDKUNDE_NR,KUERZEL,KANAL,BETREFF,STARTTERMIN
51052,656627,RAM,Besuch,Debriefing Kampagne Energie 360 Grad (250K) un...,2019-10-28
46562,635162,SUB,Besuch,Austausch,2019-10-28
749,105598,MZB,Besuch,TR: SOIREE DES PARTENAIRES POUR LES 20 ANS D'A...,2019-10-28
7356,157069,TRA,Besuch,visita per 2020,2019-10-28
35952,594875,JAN,Besuch,Sunny Fryday 2020 Ausstellung / Aussichten,2019-10-28


In [75]:
container_crm.sort_values("STARTTERMIN",ascending=True).head(5)

,ENDKUNDE_NR,KUERZEL,KANAL,BETREFF,STARTTERMIN
11849,472859,ROR,Besuch,Plakatierung 09,2009-01-05
18443,506117,RIA,E-Mail,camp. 550618,2009-01-05
18416,506017,CEG,Telefon,K: 550399 Komitee JA zu HarmoS,2009-01-05
18517,506633,JCO,E-Mail,(NEUKUNDE) SPR+ / NEW PRICING; Terminvereinbar...,2009-01-07
11096,468689,BGI,Event,Business Frühstück,2009-01-07


In [49]:
today

Timestamp('2019-10-28 12:22:37.143738')

In [87]:
crm_letzte_vbs.sample(5, random_state=42)

,ENDKUNDE_NR,letzte_VBs
28675,603257,"NAT,BIM,MAL,CKT"
39376,648963,"HKL,IWI"
22248,570267,"BIM,PAL,KEK,BAP,OSS,LSC,MAL"
13168,501775,"ROK,DTO,PHE,IWI"
40077,652497,"SAO,LIC"


In [84]:
desc_col(crm_letzte_vbs)

,DTYPE,NULLS,UNIQUE
ENDKUNDE_NR,int64,0/41295,41295
KUERZEL,object,0/41295,13681
